In [1]:
import sys

import numpy as np
import pandas as pd
import s3fs
import seaborn as sns

sys.path.append("/Users/fgu/dev/projects/mdb_eval")
import src.data.aggregators as agg
import src.data.make_data as md
import src.data.selectors as sl
import src.helpers.data as hd
import src.helpers.io as io

fs = s3fs.S3FileSystem(profile="3di")

pd.set_option("display.max_rows", 120)
pd.set_option("display.max_columns", 120)
pd.set_option("max_colwidth", None)
%load_ext autoreload
%autoreload 2

## load data

In [32]:
dfa = hd.read_analysis_data("XX0")
hd.inspect(dfa)

shape: (15,152, 32), users: 546


,user_id,ym,ymn,month,txns_count,txns_volume,month_income,inflows,outflows,netflows,netflows_norm,inflows_norm,outflows_norm,has_pos_netflows,pos_netflows,user_reg_ym,t,tt,month_spend,age,is_female,region,is_urban,region_code,has_savings_account,has_current_account,generation,generation_code,prop_credit,discret_spend,accounts_active,accounts_total
0,448040,2018-01,201801,1,29.0,11871.750000,1764.111694,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,2018-07,0,-6,1683.150024,28.0,0.0,north west,1.0,8,True,True,Millennials,3,0.162986,249.330002,2,3
1,448040,2018-02,201802,2,48.0,4696.459961,1764.111694,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,2018-07,0,-5,2274.629883,28.0,0.0,north west,1.0,8,True,True,Millennials,3,0.402202,681.099976,2,3


In [36]:
dfa.groupby()

5

In [17]:
dft = dft = io.read_parquet("s3://3di-data-mdb/clean/mdb_111.parquet")
hd.inspect(dft)

shape: (593,133, 35), users: 270


,date,user_id,amount,desc,merchant,tag_group,tag_spend,user_registration_date,account_created,account_id,account_last_refreshed,account_provider,account_type,birth_year,data_warehouse_date_created,data_warehouse_date_last_updated,id,is_debit,is_female,is_sa_flow,is_salary_pmt,is_urban,latest_balance,lsoa,merchant_business_line,msoa,postcode,region_name,salary_range,tag,tag_auto,tag_manual,tag_up,updated_flag,ym
0,2012-01-03,111,-114.0,<mdbremoved>,NaN,NaN,NaN,2010-06-19,2016-10-30,235634,2018-08-08 10:24:00,lloyds bank,current,NaN,2014-07-18,2015-03-19,397323,False,0.0,False,False,1.0,0.5,e01001036,NaN,e02000222,cr2 7,london,,NaN,NaN,no tag,no tag,u,2012-01
1,2012-01-03,111,8.5,<mdbremoved>,NaN,NaN,NaN,2010-06-19,2016-10-30,235634,2018-08-08 10:24:00,lloyds bank,current,NaN,2014-07-18,2015-03-19,397324,True,0.0,False,False,1.0,0.5,e01001036,NaN,e02000222,cr2 7,london,,NaN,NaN,no tag,no tag,u,2012-01


In [62]:
s = pd.Series(range(1, 11))
s, s.quantile(.8, interpolation='higher')

(0     1
 1     2
 2     3
 3     4
 4     5
 5     6
 6     7
 7     8
 8     9
 9    10
 dtype: int64,
 9)

In [64]:
dfa[dfa.user_id.eq(448040)].month_income

0     1764.111694
1     1764.111694
2     1764.111694
3     1764.111694
4     1764.111694
5     1764.111694
6     1764.111694
7     1764.111694
8     1764.111694
9     1764.111694
10    1764.111694
11    1764.111694
12     889.331665
13     889.331665
14     889.331665
15     889.331665
16     889.331665
17     889.331665
18     889.331665
19     889.331665
20     889.331665
21     889.331665
22     889.331665
23     889.331665
Name: month_income, dtype: float32

In [53]:
dfa.groupby("user_id").month_income.quantile(q=0.8)

user_id
448040    1764.111694
448070    1740.929199
448740    4187.071777
448910    6960.441406
449550    1680.581665
             ...     
571930    5161.002441
572280    2304.997559
573040    3647.259277
573210    1065.796265
574910    5536.148926
Name: month_income, Length: 546, dtype: float64

In [49]:
sl.year_income(dfa)

user_id
448040    1764.111694
448070    1740.929199
448740    4187.071777
448910    6960.441406
449550    1680.581665
             ...     
571930    5161.002441
572280    2304.997559
573040    3647.259277
573210    1065.796265
574910    5536.148926
Name: month_income, Length: 546, dtype: float32

# dev

In [31]:
dfa.user_reg_ym.ge("2017-04").mean()

0.9973956429563564

In [37]:
dfa.groupby("t").tt.describe().T

t,0,1
count,8574.000000,6578.000000
mean,-10.710870,6.508361
std,8.352663,4.985124
min,-35.000000,0.000000
25%,-15.000000,3.000000
50%,-8.000000,6.000000
75%,-4.000000,10.000000
max,-1.000000,24.000000


In [38]:
pd.crosstab(dfa.tt, dfa.t)

t,0,1
tt,,
-35,47,0
-34,53,0
-33,58,0
-32,65,0
-31,68,0
-30,74,0
-29,78,0
-28,86,0
-27,88,0


In [40]:
k = dfa.query("(t == 1) & (tt == 0)")[["user_id", "ym", "t", "tt", "user_reg_ym"]]
k

,user_id,ym,t,tt,user_reg_ym
6,448040,2018-07,1,0,2018-07
35,448070,2018-07,1,0,2018-07
65,448740,2018-07,1,0,2018-07
99,448910,2018-07,1,0,2018-07
119,449550,2018-07,1,0,2018-07
...,...,...,...,...,...
15009,571930,2020-02,1,0,2020-02
15050,572280,2020-02,1,0,2020-02
15091,573040,2020-02,1,0,2020-02
15105,573210,2020-02,1,0,2020-02
